In [1]:
import tensorflow as tf
import numpy as np
import time
import copy

import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append("/home/shuo/Shuo/tensorflow_models/research")
sys.path.append("/home/shuo/Shuo/tensorflow_models/research/object_detection")
from tensorflow.core.framework import graph_pb2
from utils import label_map_util
from PIL import Image

In [2]:
MODEL_ZOO = [
            'faster_rcnn_inception_resnet_v2_atrous_coco',                   # 0  .37
            'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco',      # 1  
            'faster_rcnn_inception_v2_coco',                                 # 2  .28
            'faster_rcnn_nas_coco',                                          # 3  .43
            'faster_rcnn_nas_lowproposals_coco',                             # 4  
            'faster_rcnn_resnet50_coco',                                     # 5  .30
            'faster_rcnn_resnet50_lowproposals_coco',                        # 6  
            'faster_rcnn_resnet101_coco',                                    # 7  .32
            'faster_rcnn_resnet101_kitti',                                   # 8  .87
            'faster_rcnn_resnet101_lowproposals_coco',                       # 9  
            'rfcn_resnet101_coco',                                           # 10 .30
            'ssd_inception_v2_coco',                                         # 11 .24
            'ssd_mobilenet_v1_coco'                                          # 12 .21
            ]  

USE_MODEL_ID = 12

MODEL_NAME = MODEL_ZOO[USE_MODEL_ID]

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/home/shuo/Shuo/tensorflow_models/research/object_detection/off_the_shelf_models/' + MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box. 
# PATH_TO_LABELS = os.path.join('/home/shuo/Shuo/tensorflow_models/research/object_detection/data', 'aic_label_map.pbtxt')
PATH_TO_LABELS = os.path.join('/home/shuo/Shuo/tensorflow_models/research/object_detection/data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
if MODEL_NAME[-5:] == 'kitti':
    PATH_TO_LABELS = os.path.join('/home/shuo/Shuo/tensorflow_models/research/object_detection/data', 'kitti_label_map.pbtxt')
    NUM_CLASSES = 2



In [3]:
def _node_name(n):
  if n.startswith("^"):
    return n[1:]
  else:
    return n.split(":")[0]

input_graph = tf.Graph()
# print(input_graph.as_graph_def().node)
with tf.Session(graph=input_graph):
#     print(input_graph.as_graph_def().node)
    score = tf.placeholder(tf.float32, shape=(None, 1917, 90), name="Postprocessor/convert_scores")
    expand = tf.placeholder(tf.float32, shape=(None, 1917, 1, 4), name="Postprocessor/ExpandDims_1")
#     print(input_graph.as_graph_def().node)
    for node in input_graph.as_graph_def().node:
        if node.name == "Postprocessor/convert_scores":
            score_def = node
        if node.name == "Postprocessor/ExpandDims_1":
            expand_def = node

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    dest_nodes = ['Postprocessor/convert_scores','Postprocessor/ExpandDims_1']

    edges = {}
    name_to_node_map = {}
    node_seq = {}
    seq = 0
#     print(od_graph_def.node)
    for node in od_graph_def.node:
      n = _node_name(node.name)
      name_to_node_map[n] = node
      edges[n] = [_node_name(x) for x in node.input]
      node_seq[n] = seq
      seq += 1

    for d in dest_nodes:
      assert d in name_to_node_map, "%s is not in graph" % d

    nodes_to_keep = set()
    next_to_visit = dest_nodes[:]
    while next_to_visit:
      n = next_to_visit[0]
      del next_to_visit[0]
      if n in nodes_to_keep:
        continue
      nodes_to_keep.add(n)
      next_to_visit += edges[n]

    nodes_to_keep_list = sorted(list(nodes_to_keep), key=lambda n: node_seq[n])

    nodes_to_remove = set()
#    score = tf.placeholder(tf.float32, shape=(None, 1917, 90), name="Postprocessor/convert_scores")
#    expand = tf.placeholder(tf.float32, shape=(None, 1917, 1, 4), name="Postprocessor/ExpandDims_1")

    for n in node_seq:
      if n in nodes_to_keep_list: continue
      nodes_to_remove.add(n)
    nodes_to_remove_list = sorted(list(nodes_to_remove), key=lambda n: node_seq[n])

    keep = graph_pb2.GraphDef()
    for n in nodes_to_keep_list:
      keep.node.extend([copy.deepcopy(name_to_node_map[n])])

    remove = graph_pb2.GraphDef()
    remove.node.extend([score_def])
    remove.node.extend([expand_def])
    for n in nodes_to_remove_list:
      remove.node.extend([copy.deepcopy(name_to_node_map[n])])

    with tf.device('/gpu:0'):
      tf.import_graph_def(keep, name='')
    with tf.device('/cpu:0'):
      tf.import_graph_def(remove, name='')

ops = detection_graph.get_operations()
for op in ops:
  print(op.name+' >> '+op.device)



image_tensor >> 
ToFloat >> 
Preprocessor/map/Shape >> 
Preprocessor/map/TensorArrayUnstack/Shape >> 
BoxPredictor_0/ClassPredictor/weights/read/_178__cf__181 >> 
BoxPredictor_0/ClassPredictor/biases/read/_177__cf__180 >> 
BoxPredictor_1/ClassPredictor/weights/read/_176__cf__179 >> 
BoxPredictor_1/ClassPredictor/biases/read/_175__cf__178 >> 
BoxPredictor_2/ClassPredictor/weights/read/_174__cf__177 >> 
BoxPredictor_2/ClassPredictor/biases/read/_173__cf__176 >> 
BoxPredictor_3/ClassPredictor/weights/read/_172__cf__175 >> 
BoxPredictor_3/ClassPredictor/biases/read/_171__cf__174 >> 
BoxPredictor_4/ClassPredictor/weights/read/_170__cf__173 >> 
BoxPredictor_4/ClassPredictor/biases/read/_169__cf__172 >> 
BoxPredictor_5/ClassPredictor/weights/read/_168__cf__171 >> 
BoxPredictor_5/ClassPredictor/biases/read/_167__cf__170 >> 
MultipleGridAnchorGenerator/Reshape/_163__cf__166 >> 
MultipleGridAnchorGenerator/div_14/_162__cf__165 >> 
MultipleGridAnchorGenerator/Meshgrid_2/ExpandedShape/concat/_161_

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/add_28/y >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/add_28 >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_29/Greater/y >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_29/Greater >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_29/Where >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_29/Reshape/shape >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_29/Reshape >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_29/Cast >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGrea

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Area/Squeeze >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Greater/y >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Greater >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Where >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Reshape/shape >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Reshape >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Cast >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Gather/Gather >> 
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppressi

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_49_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/stack_50/1_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/stack_50_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Slice_50/begin_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Slice_50_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_50/shape_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_50_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/stack_51/1_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxS

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_43/Reshape/shape_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_43/Reshape_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_43/Cast_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_43/Gather/Gather_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_43/Gather/Gather_2_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_43/split/split_dim_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_43/split_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while

Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_1/TensorArrayGatherV3_1 >> /device:CPU:0
detection_scores_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_2/range/start_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_2/range/delta_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_2/range_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_2/TensorArrayGatherV3_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_4/range/start_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_4/range/delta_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_4/range_1 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/TensorArrayStack_4/TensorArrayGatherV3_1 >> /device:CPU:0
Postprocessor/ToFloat_1 >> /device:CPU:0
nu

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
import tensorflow as tf
import numpy as np
import time
import copy

from tensorflow.core.framework import graph_pb2
import sys
sys.path.append("/home/shuo/Shuo/tensorflow_models/research")
sys.path.append("/home/shuo/Shuo/tensorflow_models/research/object_detection")

from utils import label_map_util
from PIL import Image

def _node_name(n):
  if n.startswith("^"):
    return n[1:]
  else:
    return n.split(":")[0]

input_graph = tf.Graph()
with tf.Session(graph=input_graph):
    score = tf.placeholder(tf.float32, shape=(None, 1917, 90), name="Postprocessor/convert_scores")
    expand = tf.placeholder(tf.float32, shape=(None, 1917, 1, 4), name="Postprocessor/ExpandDims_1")
    for node in input_graph.as_graph_def().node:
        if node.name == "Postprocessor/convert_scores":
            score_def = node
        if node.name == "Postprocessor/ExpandDims_1":
            expand_def = node

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile('/home/shuo/Shuo/tensorflow_models/research/object_detection/off_the_shelf_models/ssd_mobilenet_v1_coco/frozen_inference_graph.pb', 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    dest_nodes = ['Postprocessor/convert_scores','Postprocessor/ExpandDims_1']

    edges = {}
    name_to_node_map = {}
    node_seq = {}
    seq = 0
    for node in od_graph_def.node:
      n = _node_name(node.name)
      name_to_node_map[n] = node
      edges[n] = [_node_name(x) for x in node.input]
      node_seq[n] = seq
      seq += 1

    for d in dest_nodes:
      assert d in name_to_node_map, "%s is not in graph" % d

    nodes_to_keep = set()
    next_to_visit = dest_nodes[:]
    while next_to_visit:
      n = next_to_visit[0]
      del next_to_visit[0]
      if n in nodes_to_keep:
        continue
      nodes_to_keep.add(n)
      next_to_visit += edges[n]

    nodes_to_keep_list = sorted(list(nodes_to_keep), key=lambda n: node_seq[n])

    nodes_to_remove = set()
#    score = tf.placeholder(tf.float32, shape=(None, 1917, 90), name="Postprocessor/convert_scores")
#    expand = tf.placeholder(tf.float32, shape=(None, 1917, 1, 4), name="Postprocessor/ExpandDims_1")

    for n in node_seq:
      if n in nodes_to_keep_list: continue
      nodes_to_remove.add(n)
    nodes_to_remove_list = sorted(list(nodes_to_remove), key=lambda n: node_seq[n])

    keep = graph_pb2.GraphDef()
    for n in nodes_to_keep_list:
      keep.node.extend([copy.deepcopy(name_to_node_map[n])])

    remove = graph_pb2.GraphDef()
    remove.node.extend([score_def])
    remove.node.extend([expand_def])
    for n in nodes_to_remove_list:
      remove.node.extend([copy.deepcopy(name_to_node_map[n])])

    with tf.device('/gpu:0'):
      tf.import_graph_def(keep, name='')
    with tf.device('/cpu:0'):
      tf.import_graph_def(remove, name='')

ops = detection_graph.get_operations()
for op in ops:
  print(op.name+' >> '+op.device)

"""
tmp_graph = tf.Graph()
with tmp_graph.as_default():
  for op in ops:
    tmp_graph.create_op(op.type, op.inputs, op.inputs[0].dtypes)
  ops = tmp_graph.get_operations()
  for op in ops:
    print op.device
"""

"""
detection_graph = tf.Graph()
with detection_graph.as_default():
  tf.import_graph_def(sub_map_graph, name='')
  ops = detection_graph.get_operations()

  for op in ops:
    print op.name


ps = detection_graph.get_operations()

#    with tf.device('/cpu:0'):

    tf.import_graph_def(od_graph_def, name='')
    ops = detection_graph.get_operations()
#   map_op = [op.name for op in ops if op.name.find('Postprocessor/BatchMultiClassNonMaxSuppression/map')>=0]
#   map_op = [op.name for op in ops]
    map_op = []
    for op in ops:
      if op.name.find('Postprocessor/BatchMultiClassNonMaxSuppression/map')>=0:
        map_op.append(op.name)
    map_op.append('detection_boxes')
    map_op.append('detection_scores')
    map_op.append('detection_classes')
    map_op.append('num_detections')

    subgraph = tf.graph_util.extract_sub_graph(detection_graph.as_graph_def(), map_op)
    tf.import_graph_def(subgraph, name='')
    ops = detection_graph.get_operations()
    for op in ops:
      print op.name

NUM_CLASSES = 90
label_map = label_map_util.load_labelmap('data/mscoco_label_map.pbtxt')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)



TEST_IMAGE_PATHS = 'test_images/image1.jpg'

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
#   image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    score = detection_graph.get_tensor_by_name('Postprocessor/convert_scores:0')
    expand = detection_graph.get_tensor_by_name('Postprocessor/ExpandDims_1:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    image = Image.open(TEST_IMAGE_PATHS)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    for i in range(200):
      start_time = time.time()
#      sess.run([score, expand], feed_dict={image_tensor: image_np_expanded})
      (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={score:image_tensor: image_np_expanded})
      print 'Iteration %d: %.2f ms'%(i, time.time()-start_time)
"""

image_tensor >> /device:GPU:0
ToFloat >> /device:GPU:0
Preprocessor/map/Shape >> /device:GPU:0
Preprocessor/map/TensorArrayUnstack/Shape >> /device:GPU:0
BoxPredictor_0/ClassPredictor/weights/read/_178__cf__181 >> /device:GPU:0
BoxPredictor_0/ClassPredictor/biases/read/_177__cf__180 >> /device:GPU:0
BoxPredictor_1/ClassPredictor/weights/read/_176__cf__179 >> /device:GPU:0
BoxPredictor_1/ClassPredictor/biases/read/_175__cf__178 >> /device:GPU:0
BoxPredictor_2/ClassPredictor/weights/read/_174__cf__177 >> /device:GPU:0
BoxPredictor_2/ClassPredictor/biases/read/_173__cf__176 >> /device:GPU:0
BoxPredictor_3/ClassPredictor/weights/read/_172__cf__175 >> /device:GPU:0
BoxPredictor_3/ClassPredictor/biases/read/_171__cf__174 >> /device:GPU:0
BoxPredictor_4/ClassPredictor/weights/read/_170__cf__173 >> /device:GPU:0
BoxPredictor_4/ClassPredictor/biases/read/_169__cf__172 >> /device:GPU:0
BoxPredictor_5/ClassPredictor/weights/read/_168__cf__171 >> /device:GPU:0
BoxPredictor_5/ClassPredictor/biases/

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Greater >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Where >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Reshape/shape >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Reshape >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Cast >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Gather/Gather >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Shape_24 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_22/Gather/Gather_2 >> /devic

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Greater >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Where >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Reshape/shape >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Reshape >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Cast >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Gather/Gather >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Shape_75 >> /device:CPU:0
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_73/Gather/Gather_2 >> /devic

"\ndetection_graph = tf.Graph()\nwith detection_graph.as_default():\n  tf.import_graph_def(sub_map_graph, name='')\n  ops = detection_graph.get_operations()\n\n  for op in ops:\n    print op.name\n\n\nps = detection_graph.get_operations()\n\n#    with tf.device('/cpu:0'):\n\n    tf.import_graph_def(od_graph_def, name='')\n    ops = detection_graph.get_operations()\n#   map_op = [op.name for op in ops if op.name.find('Postprocessor/BatchMultiClassNonMaxSuppression/map')>=0]\n#   map_op = [op.name for op in ops]\n    map_op = []\n    for op in ops:\n      if op.name.find('Postprocessor/BatchMultiClassNonMaxSuppression/map')>=0:\n        map_op.append(op.name)\n    map_op.append('detection_boxes')\n    map_op.append('detection_scores')\n    map_op.append('detection_classes')\n    map_op.append('num_detections')\n\n    subgraph = tf.graph_util.extract_sub_graph(detection_graph.as_graph_def(), map_op)\n    tf.import_graph_def(subgraph, name='')\n    ops = detection_graph.get_operations()\n 